In [1]:
import gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import LatentDirichletAllocation as LDA
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')
display_action_id = pd.read_csv('../inputs/display_action_id.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [4]:
target_category = [
    38,  # アイスクリーム__ノベルティー
    110,  # スナック・キャンディー__ガム
    113,  # スナック・キャンディー__シリアル
    114,  # スナック・キャンディー__スナック
    134,  # チョコ・ビスクラ__チョコレート
    171,  # ビール系__RTD
    172,  # ビール系__ノンアルコール
    173,  # ビール系__ビール系
    376,  # 和菓子__米菓
    435,  # 大型PET__無糖茶（大型PET）
    467,  # 小型PET__コーヒー（小型PET）
    537,  # 水・炭酸水__大型PET（炭酸水）
    539,  # 水・炭酸水__小型PET（炭酸水）
    629,  # 缶飲料__コーヒー（缶）
    768,  # 麺類__カップ麺
]

In [5]:
# 2020-01-01以降、2020-08-01以前で10分以上経過し購買が発生したセッションにtrainデータを絞る
tmp_sessions = meta[(meta['date'] >= '2020-01-01')&(meta['date'] < '2020-08-01')]['session_id'].unique()
tmp_log = cartlog[cartlog["session_id"].isin(tmp_sessions)]
print('2020-01-01以降、2020-08-01以前: ', len(tmp_sessions))

# 購買が発生したセッション
payment_sessions = set(tmp_log[tmp_log['is_payment']==1]['session_id'].unique())
print('購買が発生: ', len(payment_sessions))
# 10分以上のセッション
over10min_sessions = set(tmp_log[tmp_log['spend_time']>=600]['session_id'].unique())
print('10分以上: ', len(over10min_sessions))
# 積集合
all_train_sessions = payment_sessions & over10min_sessions
print('積集合: ', len(all_train_sessions))

# 10分以上の全trainのログデータ
all_train_log = tmp_log[tmp_log["session_id"].isin(all_train_sessions)]
print('全trainのログ: ', len(all_train_log))

2020-01-01以降、2020-08-01以前:  234103
購買が発生:  229959
10分以上:  174412
積集合:  172868
全trainのログ:  4548534


In [6]:
def agg_payment(cartlog) -> pd.DataFrame:
    """セッションごと・商品ごとの購買個数を集計する"""
    # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
    agg = cartlog.loc[cartlog["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    return pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner").drop(columns=['JAN'])

In [7]:
class RetailDataset:
    def __init__(self, thres_sec, meta):
        self.thres_sec = thres_sec
        self.meta = meta.copy()
        self.meta['time_elapsed_sec'] = self.meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        
        # all_train_logの中で、指定時間より前のログのみを抽出　-> public_train_log, train_sessions
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        self.public_train_log = merge_train[merge_train['spend_time'] <= merge_train['time_elapsed_sec']]
        self.train_sessions = self.public_train_log["session_id"].unique()
        
    def get_train_input_log(self) -> pd.DataFrame:
        return self.public_train_log

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        train_target = pd.DataFrame(
            index=self.train_sessions,
        )
        train_target.index.name = "session_id"

        # time_elapsed以降のデータから購買個数を集計する
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        after_elapsed_log = merge_train[merge_train['spend_time'] > merge_train['time_elapsed_sec']]
        
        train_item_num = agg_payment(after_elapsed_log)
        train_item_num = train_item_num[train_item_num['category_id'].isin(target_category)]
        train_target_pos = train_item_num.groupby(["session_id", "category_id"])["n_items"].sum().unstack().fillna(0).astype(int)
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        return train_target.join(train_target_pos).fillna(0).reset_index()

In [8]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [9]:
ELAPSED_MIN = [0, 3, 5, 10]
# ELAPSED_MIN = [5]

In [10]:
LOG_VER = 4

In [11]:
def save_train_log(ver):
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train{}_log_{}.f'.format(ver, elap_min))
        train_y.to_feather('../inputs/train{}_y_{}.f'.format(ver, elap_min))

In [12]:
def load_train_log(ver):
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train{}_log_{}.f'.format(ver, elap_min))
        y = pd.read_feather('../inputs/train{}_y_{}.f'.format(ver, elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [13]:
# save_train_log(LOG_VER)

#### ver.4: 2020-01-01以降、2020-08-01以前
- 0, train_session (170752, 16)
- 3, train_session (172654, 16)
- 5, train_session (172794, 16)
- 10, train_session (172835, 16)

#### ver.3: 2020-04-01以降、2020-08-01以前
- 0, train_session (101562, 16)
- 3, train_session (102348, 16)
- 5, train_session (102394, 16)
- 10, train_session (102410, 16)

#### ver.2: 2020-08-01以前
- 0, train_session (378594, 16)
- 3, train_session (389649, 16)
- 5, train_session (390621, 16)
- 10, train_session (391074, 16)

### 過去のログデータ

In [14]:
payed_item = agg_payment(all_train_log)

In [15]:
def LDA_topic(df_input, topic, index, prefix):
    df_cp = df_input.set_index(index)
    lda = LDA(n_components=topic)
    lda_out = pd.DataFrame(lda.fit_transform(df_cp), index=df_cp.index).add_prefix(prefix)
    return lda_out.reset_index()

### ユーザ情報

In [16]:
user_features = pd.merge(meta[["session_id", "user_id"]], user_master, on="user_id", how="left")
user_features.loc[user_features['age'] >= 80, 'age'] = np.NaN
user_features.loc[user_features['age'] < 10, 'age'] = np.NaN
user_features.loc[user_features['gender'] > 1, 'gender'] = np.NaN

In [17]:
def get_user_item(payed_item):
    # train用のデータから購買した商品
    user_payed_item = pd.merge(payed_item, meta[['session_id', 'user_id']], on='session_id', how='left')
    group_user_item = user_payed_item.groupby(['user_id', 'category_id'])[['n_items']].sum().reset_index()
    pivot_user_item = group_user_item.pivot_table(index='user_id', columns='category_id', values='n_items')
    # 全ユーザーの購入数の合計が5000以上のカテゴリに絞り込み
    sum_user_item = pivot_user_item.sum()
    user_item_index = sum_user_item[sum_user_item > 5000].index
    pivot_user_item = pivot_user_item[user_item_index].fillna(0).reset_index()
    # trainに存在しない人用に平均値で穴埋め
    user_item_mean = pivot_user_item.mean()
    # 全ユーザーとマージ
    all_user_item = pd.merge(user_master[['user_id']], pivot_user_item, on='user_id', how='left')
    # targetのカテゴリは除く
    for col in all_user_item.columns:
        if (col == 'user_id') or (col in target_category):
            continue
        new_col = 'user_pay_{}'.format(col)
        all_user_item[new_col] = all_user_item[col].fillna(user_item_mean[col]).astype('float32')
        all_user_item.loc[all_user_item[new_col]<0, new_col] = 0

    return all_user_item.drop(columns=list(user_item_mean.index))

In [18]:
all_user_item = get_user_item(payed_item)
print(all_user_item.shape)

(40350, 153)


In [19]:
def save_item_lda(ver, all_user_item):
    user_lda = LDA_topic(all_user_item, 10, 'user_id', 'LDA_')
    user_lda.to_feather('../inputs/user_lda_{}.f'.format(ver))

In [20]:
def load_item_lda(ver):
    user_lda = pd.read_feather('../inputs/user_lda_{}.f'.format(ver))
    return user_lda

In [21]:
# %%time
# save_item_lda(LOG_VER, all_user_item)

In [22]:
user_lda = load_item_lda(LOG_VER)
all_user_item = pd.merge(all_user_item, user_lda, on='user_id', how='left')
print(all_user_item.shape)

(40350, 163)


### メタ情報

In [23]:
def get_meta_features(meta):
    meta_features = meta.copy()
    meta_features['year'] = meta_features['date'].dt.year
    meta_features['month'] = meta_features['date'].dt.month
    meta_features['day'] = meta_features['date'].dt.day
    meta_features['dow'] = meta_features['date'].dt.dayofweek
#     meta_features['doy'] = meta_features['date'].dt.dayofyear
    meta_features['week_time'] = meta_features['dow'] * 24 + meta_features['hour']
    le = preprocessing.LabelEncoder()
    meta_features['userid'] = le.fit_transform(meta_features['user_id'])
    
    # 曜日x時間の来店者数
    df_tz = meta_features.groupby(['week_time']).size().rename('timezone_count')
    df_tz =  pd.DataFrame(df_tz).reset_index()
    meta_features = pd.merge(meta_features, df_tz, on='week_time', how='left')
    
    # userごとに前の来店からどれくらいの日数が経ったか
    df = meta_features.groupby(['session_id', 'user_id', 'date']).first().reset_index().sort_values(['user_id', 'session_id'])
    df['date_diff'] = df['date'].diff(1)
    df['user_diff'] = df['user_id'].shift(1)
    df.loc[df['user_diff'] != df['user_id'], 'date_diff'] = pd.NaT
    df['date_diff'] = df['date_diff'].dt.days
    meta_features = meta_features.join(df[['date_diff']])

    # userごとの来店回数
    meta_features['date_rank'] = meta_features.groupby(['user_id'])['date'].rank(ascending=True)
    
    return meta_features.drop(columns=['user_id', 'date', 'time_elapsed', 'date_str'])

In [24]:
meta_features = get_meta_features(meta)

### ディスプレイアクション

In [25]:
disp_name_dic = {}
for i, disp in enumerate(display_action_id['display_name'].unique()):
    disp_name_dic[disp] = 'disp_cnt_{}'.format(i)
    
act_name_dic = {}
for i, action in enumerate(display_action_id['action_name'].unique()):
    act_name_dic[action] = 'act_cnt_{}'.format(i)

In [26]:
def get_display_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    disp_group_count = merge.groupby(['session_id', 'display_name']).size().reset_index().rename(columns={0:'disp_name_count'})
    disp_name_pivot = disp_group_count.pivot_table(index='session_id', columns='display_name', values='disp_name_count', aggfunc='sum')
    disp_name_pivot = disp_name_pivot.reset_index().fillna(0).rename(columns=disp_name_dic)
    
    disp_out = disp_name_pivot[['session_id']].copy()
    for val in disp_name_dic.values():
        disp_out[val] = 0
    
    for col in disp_name_pivot.columns:
        if col == 'session_id':
            continue
        disp_out[col] = disp_name_pivot[col]
    
    return disp_out

In [27]:
def get_action_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    act_group_count = merge.groupby(['session_id', 'action_name']).size().reset_index().rename(columns={0:'act_name_count'})
    act_name_pivot = act_group_count.pivot_table(index='session_id', columns='action_name', values='act_name_count', aggfunc='sum')
    act_name_pivot = act_name_pivot.reset_index().fillna(0).rename(columns=act_name_dic)
    
    act_out = act_name_pivot[['session_id']].copy()
    for val in act_name_dic.values():
        act_out[val] = 0
    
    for col in act_name_pivot.columns:
        if col == 'session_id':
            continue
        act_out[col] = act_name_pivot[col]
    
    return act_out

### セッション単位の特徴量

In [28]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'value_1']].rename(columns={'value_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, lda_coupon, on='coupon', how='left').drop(columns=['coupon'])
#     session_coupon = session_coupon.groupby(['session_id']).max().reset_index()
#     return session_coupon

In [29]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'name_1']].rename(columns = {'name_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, df_coupon_cat, on='coupon', how='left').drop(columns=['coupon'])
    
#     for cat in df_coupon_cat['coup_cat'].unique():
#         new_col = 'coup_cat_{}'.format(cat)
#         session_coupon[new_col] = 0
#         session_coupon.loc[session_coupon['coup_cat'] == cat, new_col] = 1
    
#     session_coupon.drop(columns=['coup_cat'], inplace=True)
#     return session_coupon.groupby('session_id').sum().reset_index()

In [30]:
def get_pre_payment_item(input_log):
    session_unique = input_log['session_id'].unique()
    agg = input_log.loc[input_log["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    agg = pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    agg = agg[agg['category_id'].isin(target_category)]
    agg = agg.groupby(["session_id", "category_id"])["n_items"].sum().reset_index()
    
    sesi = np.zeros(len(target_category))
    cate = [ct for ct in target_category]
    
    dummy = pd.DataFrame({'session_id':sesi, 'category_id':cate, 'n_items':sesi})
    agg = pd.concat([agg, dummy])
    
    agg = agg.pivot_table(index='session_id', columns='category_id', values='n_items').fillna(0)
    src_columns = ['x_{}'.format(c) for c in agg.columns]
    agg.columns = src_columns
    
    col = ['pre_target_{}'.format(c) for c in target_category]
    df_out = pd.DataFrame(index=session_unique, columns=col)
    df_out.index.name = "session_id"
    df_out = df_out.join(agg)
    for ct in target_category:
        src = 'x_{}'.format(ct)
        dst = 'pre_target_{}'.format(ct)
        df_out[dst] = df_out[src]
    
    return df_out.drop(columns=src_columns).fillna(0).reset_index()

In [31]:
def get_session_kind_group(input_log):
    kind_name ={
        'クーポン': 'coupon',
        '会計': 'kaikei',
        'キー': 'key',
        'カテゴリ': 'categry',
        'バーコードスキャン': 'barcode',
        'UUID': 'uuid',
        '使用ポイント': 'usedpoint',
        '確認': 'confirm',
        'ブランドスイッチ': 'bland',
        'レシピ': 'recipe',
        'スマホスキャン': 'smartphone',
        '磁気スキャン': 'magnetic',
        'レコメンド': 'recommend',
        '倍率ポイント': 'point',
    }
    group_count = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename('group_count_'+'item')
    for kind, name in kind_name.items():
        tmp = input_log[input_log['kind_1'] == kind].groupby(["session_id"]).size().rename('group_count_'+name)
        group_count = pd.concat([group_count, tmp], axis=1)
        
    return group_count.reset_index()

In [32]:
def get_session_item_info(input_log):
    item_log = input_log[input_log['kind_1'] == '商品'].copy()
    item_log = item_log.rename(columns={"value_1": "JAN"})
    item_log = pd.merge(item_log, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    item_log['total'] = item_log['n_items'] * item_log['unit_price']
    session_item = item_log.groupby(['session_id']).agg({
        'total':'sum', 
        'number_1':'sum', 
        'n_items':'sum', 
        'name_1':'nunique',
        'category_id':'nunique',
    }).reset_index()
    session_item = session_item.rename(columns={
        'number_1':'cart_item_cnt', 
        'n_items':'total_item_cnt', 
        'name_1':'item_nunique',
        'category_id':'cat_nunique',
    })
    session_item['mean_price'] = session_item['total'] / session_item['total_item_cnt']
    session_item['item_cnt_per_nuniq'] = session_item['total_item_cnt'] / session_item['item_nunique']
    session_item['item_nuniq_per_cat_nuniq'] = session_item['item_nunique'] / session_item['cat_nunique']
    session_item['mean_price_per_cat'] = session_item['total'] / session_item['cat_nunique']
    return session_item

In [33]:
def get_session_info(input_log):
    # アクション数
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    # 経過時間の平均
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()
    # ユニークユーザー
    unique_user = input_log.groupby(["session_id"])['user_id'].nunique().rename("uniq_user")
    
    session_features = pd.concat([
        n_actions,
        mean_spend_time,
        unique_user,
    ], axis=1)
    
    session_features['n_actions_user'] = session_features['n_actions'] * session_features['uniq_user']
    session_features['spend_time_user'] = session_features['spend_time'] * session_features['uniq_user']
    
    return session_features.reset_index()

### セッション単位で集計

In [34]:
def get_session_features(input_log):
    df_ses = pd.DataFrame(columns=['session_id'])
    
    session_feat = [
        get_session_info(input_log),
        get_session_kind_group(input_log),
        get_display_name_feature(input_log),
        get_action_name_feature(input_log),
        get_pre_payment_item(input_log),
#         get_coupon_info(input_log),
        get_session_item_info(input_log),
    ]
    
    for feat in session_feat:
        df_ses = pd.merge(df_ses, feat, on='session_id', how='outer')
        
    return df_ses

### 特徴量を集約する

In [35]:
def merge_features(input_log, session):
    feat_list = [
        get_session_features(input_log),
        user_features,
        meta_features,
    ]
    out = pd.DataFrame({"session_id": session})
    for feat in feat_list:
        out = pd.merge(out, feat, on="session_id", how="left")
        
    # userの情報
    out = pd.merge(out, all_user_item, on='user_id', how='left').drop(columns='user_id')

    assert len(session) == len(out)
    return out

In [36]:
def get_train_all_features(elapsed_min, train_log_list, train_y_list):
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    
    train_features = merge_features(train_input_log, y_train['session_id'])
    print('train_features', train_features.shape)
    return train_features, y_train

In [37]:
def get_test_all_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    
    test_features = merge_features(test_input_elapsed, test_meta['session_id'])
    print('test_features', test_features.shape)
    return test_features

In [38]:
n_fold = 4

In [39]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
#   'max_depth': 6,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

In [40]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    cat_feat = ['age', 'gender', 'dow', 'register_number']

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train, categorical_feature = cat_feat)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train, categorical_feature = cat_feat)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    categorical_feature = cat_feat,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [41]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [42]:
train_log_list, train_y_list = load_train_log(LOG_VER)

In [43]:
gc.collect()

78

In [44]:
%%time
df_pred_all = pd.DataFrame()
df_score_all = pd.DataFrame(index=ELAPSED_MIN)
models_list_list = []

for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_all_features(elapsed_min, train_log_list, train_y_list)
    test_features = get_test_all_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    train_features.drop(columns=['session_id'], inplace=True)
    test_features.drop(columns=['session_id'], inplace=True)
    
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
        df_score_all.loc[elapsed_min, target] = score
        
    models_list_list.append(models_list)
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_features (170752, 266)
test_features (14277, 266)
---- id = 38 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[861]	valid_0's auc: 0.799003
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1013]	valid_0's auc: 0.797279
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[660]	valid_0's auc: 0.799655
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[712]	valid_0's auc: 0.805315
--- FINISHED \ whole score: 0.7996 ---
---- id = 110 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[270]	valid_0's auc: 0.765394
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[145]	valid_0's auc: 0.753529
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[567]	valid_0's auc: 0.745415
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[202]	valid_0's auc: 0.769552
--- FINISHED \ whole score: 0.7410 ---
---- id = 113 -----


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[991]	valid_0's auc: 0.802727
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[378]	valid_0's auc: 0.765025
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[353]	valid_0's auc: 0.790594
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[665]	valid_0's auc: 0.789779
--- FINISHED \ whole score: 0.7754 ---
---- id = 629 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[208]	valid_0's auc: 0.829658
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1323]	valid_0's auc: 0.831741
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[240]	valid_0's auc: 0.828262
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[257]	valid_0's auc: 0.760619
--- FINISHED \ whole score: 0.7454 ---
---- id = 768 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[342]	valid_0's auc: 0.687726
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[192]	valid_0's auc: 0.688552
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[292]	valid_0's auc: 0.691149
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[325]	valid_0's auc: 0.687302
--- FINISHED \ whole score: 0.6885 ---
56486
CPU times: user 13h 9min 49s, sys: 6h 51min 8s, total: 20h 58s
Wall time: 53min 34s


In [45]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.130278,0.015487,0.024922,0.340639,0.056015,0.009433,0.000083,0.059449,0.190886,0.001666,0.015879,0.000901,0.004351,0.002357,0.048880
663761,0.127055,0.165856,0.308447,0.374056,0.396531,0.106626,0.081513,0.035056,0.324870,0.004644,0.004092,0.001296,0.096687,0.000912,0.500503
663763,0.091887,0.027001,0.010594,0.870958,0.448602,0.005885,0.002674,0.004084,0.236687,0.014745,0.040188,0.000493,0.008927,0.007170,0.052843
663775,0.025944,0.011485,0.021400,0.192520,0.231112,0.034022,0.006429,0.122968,0.063150,0.272141,0.050411,0.003835,0.005328,0.002579,0.203665
663778,0.123391,0.038070,0.077600,0.192155,0.124976,0.034624,0.005622,0.074266,0.036873,0.067173,0.004702,0.003347,0.017787,0.000275,0.042661


In [46]:
df_score_all

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0.799609,0.740961,0.758759,0.739014,0.721915,0.841546,0.839214,0.837718,0.734202,0.853207,0.846398,0.882674,0.830806,0.865504,0.716056
3,0.791806,0.744915,0.747857,0.732673,0.714964,0.826801,0.828695,0.823767,0.728468,0.834469,0.829567,0.865328,0.810632,0.845219,0.700847
5,0.790668,0.724082,0.743420,0.727794,0.710632,0.817922,0.814145,0.815111,0.724824,0.817375,0.823196,0.852179,0.806541,0.827070,0.690236
10,0.786878,0.716499,0.741986,0.732863,0.723617,0.798395,0.783718,0.805149,0.738800,0.803902,0.805079,0.847857,0.775391,0.745446,0.688490


In [47]:
cv = df_score_all.mean(axis=1)
print(cv)
print('- cv =', cv.mean())

0     0.800506
3     0.788401
5     0.779013
10    0.766271
dtype: float64
- cv = 0.7835476153153277


In [48]:
assert len(df_pred_all) == len(test)

In [49]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [50]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)

#### baseline_15: 2020-01-01以降, doyを除外, register_numberをcategoryに
- feat = 266
- Wall time: 53min 34s
- cv = 0.78354 (0.800/0.788/0.779/0.766)
- LB = 0.7333

#### baseline_14: 2020-01-01以降
- feat = 267
- Wall time: 36min 38s
- cv = 0.80381 (0.819/0.807/0.799/0.788)
- LB = 0.7473

#### baseline_13: 2020-04-01以降
- feat = 208
- Wall time: 25min 3s
- cv = 0.78581 (0.803/0.788/0.779/0.772)
- LB = 0.7304

#### baseline_12: 特徴量削減, lr=0.05
- feat = 345
- Wall time: 2h 18min 19s
- cv = 0.82506 (0.835/0.826/0.821/0.816)
- LB = 0.7675

#### baseline_11: クーポンカテゴリの種類減らす, ユニークユーザー数, 平均単価, ユニーク商品数 
- feat = 375
- Wall time: 2h 18min 19s
- cv = 0.81843 (0.831/0.820/0.814/0.806)
- LB = 0.7626

#### baseline_10: クーポンカテゴリの表示回数
- feat = 412
- Wall time: 2h 8min 53s
- cv = 0.821003 (0.831/0.823/0.818/0.8103)
- LB = 0.7657

#### baseline_9-1: 'feature_fraction': 0.6,'bagging_fraction': 0.6,'bagging_freq': 2,
- feat = 333
- Wall time: 2h 2min 14s
- cv = 0.81961 (0.829/0.822/0.816/0.809)
- LB = 0.7658

#### baseline_9: 来店間隔、回数、曜日x時間の来店者数
- feat = 333
- Wall time: 2h 9min 51s
- cv = 0.82118 (0.831/0.824/0.818/0.810)
- LB = 0.7665

#### baseline_8: クーポン発行とターゲットの購買実績
- feat = 344
- Wall time: 2h 5min 13s
- cv = 0.820398 (0.832/0.822/0.817/0.809)
- LB = 0.7484

#### baseline_7: userごとのカテゴリの過去の購買実績
- feat = 329
- Wall time: 1h 59min 16s
- cv = 0.82148 (0.832/0.824/0.818/0.810)
- LB = 0.7671

#### baseline_6: sesssion内のターゲットの購買
- feat = 89
- Wall time: 37min 47s
- cv = 0.66015
- LB = 0.6247

#### baseline_5
- feat = 74
- Wall time: 39min 18s
- cv = 0.66859
- LB = 0.6230

In [51]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [52]:
models = models_list_list[2]

In [53]:
feature_importance(models[0]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1354,1926,2011,1718,7009
262,date_rank,169,299,319,253,1040
263,total,171,289,318,251,1029
254,mean_price_per_cat,131,308,318,252,1009
261,mean_price,166,298,274,250,988
259,date_diff,143,264,285,258,950
245,spend_time,109,247,288,225,869
255,week_time,133,237,270,198,838
260,userid,145,241,231,188,805
248,timezone_count,116,192,264,200,772


In [54]:
feature_importance(models[1]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,426,527,760,1113,2826
263,mean_price,102,149,230,390,871
261,total,95,156,231,388,870
259,mean_price_per_cat,83,126,206,418,833
260,date_diff,93,141,217,363,814
262,date_rank,98,116,207,336,757
255,week_time,64,129,187,343,723
257,spend_time,73,112,174,319,678
256,userid,70,106,169,272,617
254,timezone_count,61,96,160,277,594


In [55]:
feature_importance(models[2]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,968,1176,996,1219,4359
262,mean_price,199,259,218,247,923
263,date_diff,202,249,191,240,882
261,total,165,252,209,240,866
260,date_rank,164,267,191,230,852
257,mean_price_per_cat,146,226,189,260,821
259,spend_time,161,228,188,226,803
256,week_time,145,214,173,204,736
255,timezone_count,137,188,181,195,701
258,userid,146,181,141,191,659


In [56]:
feature_importance(models[3]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,2320,2320,2316,2275,9231
263,date_rank,322,301,323,317,1263
261,mean_price,266,275,285,253,1079
262,mean_price_per_cat,274,250,286,241,1051
259,date_diff,253,236,264,277,1030
260,total,266,232,260,222,980
258,spend_time,234,195,215,233,877
248,userid,182,204,203,208,797
255,LDA_4,191,172,217,207,787
250,timezone_count,187,179,198,201,765


In [57]:
feature_importance(models[4]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1468,2020,1549,1452,6489
263,date_rank,204,301,233,201,939
261,date_diff,183,258,163,183,787
262,mean_price,184,256,160,185,785
260,total,172,260,154,155,741
259,mean_price_per_cat,147,251,148,154,700
257,spend_time,133,242,140,135,650
253,week_time,119,212,129,135,595
258,LDA_6,135,215,117,115,582
256,user_pay_135,132,170,144,122,568


In [58]:
feature_importance(models[5]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,2130,2046,3218,2110,9504
263,date_rank,283,275,541,296,1395
262,date_diff,281,268,525,268,1342
259,mean_price_per_cat,264,238,555,281,1338
258,mean_price,263,243,537,267,1310
261,userid,270,245,503,274,1292
260,total,264,238,534,237,1273
257,spend_time,250,214,444,224,1132
255,LDA_6,237,242,418,222,1119
256,LDA_0,243,207,390,233,1073


In [59]:
feature_importance(models[6]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1356,1115,1197,954,4622
263,total,421,264,346,213,1244
262,mean_price,396,255,357,225,1233
261,date_diff,375,276,340,201,1192
260,mean_price_per_cat,374,261,336,206,1177
259,date_rank,339,237,291,199,1066
257,spend_time,327,216,292,183,1018
258,userid,330,212,271,181,994
255,week_time,316,216,257,161,950
256,timezone_count,325,171,265,166,927


In [60]:
feature_importance(models[7]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,3700,3341,2237,3906,13184
263,mean_price,734,544,296,738,2312
261,mean_price_per_cat,646,557,260,708,2171
262,total,679,523,277,672,2151
260,date_diff,597,569,275,709,2150
259,date_rank,558,528,286,626,1998
257,userid,535,467,278,612,1892
258,spend_time,539,466,254,588,1847
255,timezone_count,487,412,216,586,1701
256,week_time,499,439,207,517,1662


In [61]:
feature_importance(models[8]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1220,1496,1336,1334,5386
263,date_rank,179,232,207,218,836
261,mean_price,171,226,197,192,786
262,date_diff,177,220,181,189,767
260,mean_price_per_cat,136,198,180,175,689
255,total,114,189,188,170,661
259,userid,131,164,141,129,565
258,spend_time,127,156,123,151,557
248,week_time,87,150,134,138,509
257,timezone_count,117,160,107,123,507


In [62]:
feature_importance(models[9]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,2841,1273,1072,1796,6982
262,date_diff,695,224,162,390,1471
263,mean_price,718,224,152,349,1443
259,date_rank,611,242,216,360,1429
260,total,670,218,169,339,1396
261,mean_price_per_cat,673,214,160,342,1389
257,spend_time,582,174,139,317,1212
258,week_time,582,161,156,295,1194
256,userid,548,184,142,275,1149
252,LDA_5,470,183,136,259,1048


In [63]:
feature_importance(models[10]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1259,1253,976,847,4335
263,date_diff,332,367,236,210,1145
262,mean_price,329,332,263,204,1128
260,mean_price_per_cat,286,345,251,148,1030
261,total,318,332,188,182,1020
258,date_rank,269,273,214,193,949
257,userid,249,279,223,156,907
259,week_time,273,281,189,160,903
256,spend_time,247,275,171,165,858
255,timezone_count,239,283,175,146,843


In [64]:
feature_importance(models[11]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1828,1816,2430,1637,7711
263,mean_price,433,474,782,378,2067
260,total,407,452,745,386,1990
261,mean_price_per_cat,421,423,726,379,1949
262,date_diff,429,430,701,330,1890
259,date_rank,397,393,681,381,1852
258,spend_time,394,401,608,339,1742
257,userid,338,347,580,307,1572
256,week_time,321,348,605,295,1569
255,timezone_count,316,350,632,255,1553


In [65]:
feature_importance(models[12]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1616,1699,1398,2230,6943
263,mean_price,365,338,292,599,1594
261,mean_price_per_cat,343,344,295,585,1567
262,date_diff,364,344,291,544,1543
260,date_rank,323,349,288,508,1468
258,total,311,361,277,505,1454
259,week_time,319,301,238,461,1319
257,spend_time,290,296,277,441,1304
256,userid,287,293,242,418,1240
254,timezone_count,240,273,239,402,1154


In [66]:
feature_importance(models[13]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,876,938,1530,754,4098
262,total,281,257,652,236,1426
263,mean_price,291,271,626,235,1423
261,mean_price_per_cat,242,263,617,195,1317
259,date_diff,238,275,530,193,1236
260,date_rank,238,238,500,202,1178
258,week_time,219,252,464,165,1100
257,timezone_count,209,204,483,179,1075
256,spend_time,188,216,469,152,1025
254,day,169,214,438,155,976


In [67]:
feature_importance(models[14]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
264,register_number,1150,1631,1334,1377,5492
262,mean_price,144,253,216,214,827
263,date_rank,155,234,189,209,787
261,date_diff,128,265,192,184,769
255,mean_price_per_cat,101,294,162,166,723
260,total,123,253,168,174,718
253,spend_time,97,220,151,153,621
259,week_time,106,209,138,161,614
256,timezone_count,104,203,128,144,579
250,userid,85,205,153,121,564
